In [6]:
# Module Importations
from sklearn.model_selection import train_test_split

In [1]:
# Project Module Importations
from src.data import load_data
from src.data import split_data
from src.models import keras_helpers

Using TensorFlow backend.


In [2]:
# Load dataset
original_dataset = load_data.load_motor_data()

In [3]:
# Split data into training / evaluation sets
training_set, evaluation_set = split_data.split_train_test(original_dataset, 0.8)

In [4]:
# Drop profile id data column
training_set = training_set.drop("profile_id", axis = 1)
print(training_set.describe())

ambient        coolant            u_d            u_q  \
count  199614.000000  199614.000000  199614.000000  199614.000000   
mean       -0.005043       0.005308       0.003792      -0.005359   
std         0.996490       1.002488       0.998747       1.001464   
min        -5.239872      -1.270450      -1.654002      -1.852853   
25%        -0.601386      -1.037872      -0.831843      -0.924537   
50%         0.266671      -0.178200       0.267391      -0.098490   
75%         0.686838       0.673512       0.358587       0.848608   
max         2.954662       2.296845       2.273808       1.788773   

         motor_speed         torque            i_d            i_q  \
count  199614.000000  199614.000000  199614.000000  199614.000000   
mean       -0.004852      -0.001687       0.004184      -0.001570   
std         1.001954       0.998525       0.999021       0.998389   
min        -1.353747      -3.339106      -3.235659      -3.329749   
25%        -0.951892      -0.267419      -0.75

In [5]:
# Create rotor target
rotor_training_data = training_set.drop("pm", axis = 1)
rotor_label_data = training_set["pm"].copy()

In [8]:
# Create train and test arrays
X_train, X_test, y_train, y_test = train_test_split(rotor_training_data, rotor_label_data, test_size = 0.2, random_state = 0)

ambient   coolant       u_d       u_q  motor_speed    torque  \
226387 -1.413809 -1.057052 -0.467056  1.478700     1.834744 -0.004685   
420886  0.635738  2.199731 -0.389888 -0.858250    -0.900093  2.673872   
109412 -0.829217 -1.075907  0.296190  1.744337     0.265571 -0.265274   
288816 -0.565807  0.301264 -0.190638 -0.825263    -0.951901  1.938591   
2472   -0.628137 -1.042617  0.183006  1.684369     2.024118 -0.264763   
...          ...       ...       ...       ...          ...       ...   
127838  0.504238  1.009462 -0.939062  0.885982    -0.004974  0.801617   
695834  0.372681  0.580977  0.302005 -1.298735    -1.222431 -0.255640   
158264 -0.084264 -1.073956 -1.091103  0.853113    -0.004971  0.931768   
548826  1.569962 -0.053349  0.239269  1.758611     0.815972 -0.250909   
591460  1.369382 -0.140655 -1.549056 -0.477226     0.738642  0.997745   

             i_d       i_q  stator_yoke  stator_tooth  stator_winding  
226387 -0.782285 -0.018301    -0.769430     -0.437997       

In [ ]:

    # Make predictions on the test data.
    y_pred = model.predict(X_test)

    mae = mean_absolute_error(y_test, y_pred)
    print("mlp nn MAE: " + str(mae))

    rmse = mean_squared_error(y_test, y_pred, squared = False)
    print("mlp nn MSE: " + str(rmse))